In [1]:
import os
import pyodbc
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely import wkt
from pymssql import connect
from great_tables import GT


In [2]:
pd.options.display.float_format = '{:.5f}'.format

vintage_year = 2025
years = list(range(2023, 2026))
# years = [2011, 2015]
geoid_col = 'GEOID20'
publication_id = '21'

# open shapefiles in output dir
out_dir = r'J:\OtherData\OFM\SAEP\SAEP Extract_2025-11-07\parcelized'

In [3]:
def sqlconn(dbname):
    # create Elmer connection
    conn_string = "DRIVER={ODBC Driver 17 for SQL Server}; SERVER=SQLserver; DATABASE=Elmer; trusted_connection=yes"
    con = pyodbc.connect(conn_string)
    return(con)

def query_tblOfmSaep(years, publication_id):
    # retrieve ofm data from Elmer
    con = sqlconn('Elmer')
    table_name = 'ofm.estimate_facts'
    min_y = str(min(years))
    max_y = str(max(years))
    # get min and max years
    query = ("SELECT a.publication_dim_id, a.estimate_year, HU = SUM(a.housing_units), OHU = SUM(a.occupied_housing_units), GQPOP = SUM(a.group_quarters_population), HHPOP = SUM(a.household_population) "
            "FROM ") + table_name + (" AS a "
            "WHERE a.publication_dim_id = ") + publication_id + (" AND a.estimate_year BETWEEN ") + min_y + (" AND ") + max_y + (" GROUP BY a.estimate_year, a.publication_dim_id;")
    df = pd.read_sql(query, con)
    con.close()
    return(df)


In [4]:
# QC parcelizer output
df = pd.DataFrame()

for year in years:
    out_file = 'parcelized_ofm_' + str(year) + '_vintage_' + str(vintage_year) + '.shp'
    print('Reading shapefile')
    shp = gpd.read_file(os.path.join(out_dir, out_file))

    # summarise where headers start with parcel_
    sum_cols = [col for col in shp if col.startswith('parcel_')]
    sum_df = pd.DataFrame(shp[sum_cols].sum())
    sum_df = sum_df.rename(columns={0:"est"})
    sum_df['est_year'] = str(year)
    sum_df = sum_df.reset_index(names=['est_type'])
    sum_df = sum_df.pivot(index='est_year', values='est', columns='est_type')
    sum_df = sum_df.reset_index()
    sum_df = sum_df.rename_axis(None, axis=1)

    if df.empty:
        df = sum_df
    else:
        df = pd.concat([df, sum_df])

print(df)

Reading shapefile
Reading shapefile
Reading shapefile
  est_year   parcel_gq    parcel_hhp     parcel_hu    parcel_ohu    parcel_tot
0     2023 80640.00195 4356459.99896 1846140.00003 1735499.74192 4437100.00091
0     2024 78632.00098 4405267.99908 1877578.99976 1759579.03917 4483900.00006
0     2025 81135.00086 4453164.99875 1907572.99984 1781459.03963 4534299.99962


In [5]:
GT(df)

est_year,parcel_gq,parcel_hhp,parcel_hu,parcel_ohu,parcel_tot
2023,80640.0019467473,4356459.9989623455,1846140.0000251827,1735499.741915772,4437100.000909095
2024,78632.00097733736,4405267.999081196,1877578.9997620587,1759579.039165872,4483900.000058534
2025,81135.00086474422,4453164.998752162,1907572.999842048,1781459.0396275434,4534299.999616904


In [6]:
# compare with Elmer results
x = query_tblOfmSaep(years, publication_id)
x = x[['publication_dim_id', "estimate_year", "GQPOP", "HHPOP", "HU", "OHU"]]
print(x)

   publication_dim_id  estimate_year       GQPOP         HHPOP            HU  \
0                  21           2023 80640.00195 4356459.99896 1846140.00003   
1                  21           2024 78632.00098 4405267.99908 1877578.99976   
2                  21           2025 81135.00086 4453164.99875 1907572.99984   

            OHU  
0 1735499.74192  
1 1759579.03917  
2 1781459.03963  


<positron-console-cell-6>:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


In [7]:
GT(x)

publication_dim_id,estimate_year,GQPOP,HHPOP,HU,OHU
21,2023,80640.0019467473,4356459.998962346,1846140.000025183,1735499.7419157727
21,2024,78632.00097733736,4405267.999081195,1877578.9997620583,1759579.0391658708
21,2025,81135.00086474419,4453164.998752162,1907572.9998420477,1781459.0396275432


In [8]:
# compare df (parcelized estimates) and x (elmer)
df1 = df.copy()
df1['est_year'] = np.int64(df1['est_year'])
df2 = x.copy()
dfs = df1.merge(df2, left_on='est_year', right_on='estimate_year')

dfs['diff_gq'] = dfs['parcel_gq'] - dfs['GQPOP']
dfs['diff_hhpop'] = dfs['parcel_hhp'] - dfs['HHPOP']
dfs['diff_hu'] = dfs['parcel_hu'] - dfs['HU']
dfs['diff_ohu'] = dfs['parcel_ohu'] - dfs['OHU']

# parcelized subtract elmerdb (original)
dfs_diff = dfs[['est_year', 'diff_gq', 'diff_hhpop', 'diff_hu', 'diff_ohu']]

print(dfs_diff)

   est_year  diff_gq  diff_hhpop  diff_hu  diff_ohu
0      2023  0.00000    -0.00000 -0.00000  -0.00000
1      2024  0.00000     0.00000  0.00000   0.00000
2      2025  0.00000     0.00000  0.00000   0.00000


In [9]:
GT(dfs_diff)

est_year,diff_gq,diff_hhpop,diff_hu,diff_ohu
2023,0.0,-9.313225746154785e-10,-2.3283064365386963e-10,-6.984919309616089e-10
2024,0.0,9.313225746154785e-10,4.656612873077393e-10,1.1641532182693481e-09
2025,2.9103830456733704e-11,0.0,2.3283064365386963e-10,2.3283064365386963e-10


In [ ]:
file_path = r'J:\OtherData\OFM\SAEP\SAEP Extract_2025-11-07\quality_check\qc-parcelized-estimates.xlsx'

with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='parcelized', index=False)
    x.to_excel(writer, sheet_name='elmer', index=False)
    dfs_diff.to_excel(writer, sheet_name='diff', index=False)